## Notebook para Execução da reconfiguração parcial com 1 core

### Elaborado por: Téo Sobrino Alves
### Feito sob licença GPL 3.0
#### Mais detalhes: https://github.com/TeOSobrino/IC_code/tree/main

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq import Overlay
import time

In [2]:
# carrega o overlay base da fpga - permite interagir com leds, botões, etc
base = BaseOverlay("base.bit")
base_path = '/home/xilinx/'
current_ol = 'none'

In [2]:
# carrega o bitstream base principal do processador
def load_main_ol():
    ol = Overlay(base_path + 'xilinx_zcu104_main.bit')

In [1]:
# devemos declarar qual região da FPGA foi marcada como reconfigurável com o 
# nome do bdf parcial usado naquele design, em nosso design temos 3 blocos

# bloco que reconfigura apenas a memória, aumentando a cache
def set_pr_mem_inc():
    load_main_ol()
    Overlay.set_partial_region('block_mem')
    current_ol = "mem_inc"

# bloco que reconfigura apenas a memória, diminuindo a cache
def set_pr_mem_inc():
    load_main_ol()
    Overlay.set_partial_region('block_mem')
    current_ol = "mem_dec"

#resetamos a arquitetura para a estática inicial
def reset_arch():
    load_main_ol()
    current_ol = None


In [ ]:
# precisamos direcionar o processador para o arquivo RISCV que executaremos (testbench)
path = "home/Xilinx/teo/tb/asm/drnn"
found = ol.bp_set_program_path(path)
if not found:
    print(f"Arquivo não encontrado em (File not found at): {path}")
    exit

In [ ]:
#lemos a taxa de hit/miss do processador BlackParrot
trigger = ol.bp_read_register('cache_rate')

In [ ]:
# podemos designar um botão para realizar a troca dos bitstream:
trigger_val = 0.7
def load_ol_trigger(program_status):
    while program_status:
        if trigger >= trigger_val:
            if current_ol != "mem_inc":
                set_pr_mem_inc()
            Overlay.download(base_path + 'black_parrot_zcu104_increased_cache.bit')
        elif trigger <= (1-trigger_val):
            if current_ol != "mem_dec":
                set_pr_mem_inc()
            Overlay.download(base_path + 'black_parrot_zcu104_reduced_cache.bit')
        # esperamos para realizar a leitura do registrador
        time.sleep(0.005)
        trigger = ol.bp_read_register('cache_rate')

        

In [ ]:
# o programa inicia sua execução
program_status = 0
ol.bp_begin_execution(program_status)

# carregamos o overlay
load_ol_trigger(program_status)